# Step 1 - Benchmarking

### Domain and Data

Here I am working with the Madelon data set, a synthetic data set with many variables and a high degree of non-linearity.  My goal is to generate an effective benchmark for the performance of logistic regression on the data set using multiple metrics.  Given the size of the data set, addition of additional features is computationally impractical, so I restrict my attention to the baseline data set.

### Problem Statement

I wish to separate the entire data set into two set sets, with approximately a 70%/30% split with randomized indices.  The larger data set will be used as a training set for training and cross-validating potential models and the latter of will be set aside as an ultimate validation set.  The validation set will not be touched till the end of step 3.  I also wish to normalize the columns in the training set and perform an identical shift and rescaling on the validation set.  Then I will produce summary metrics for a naive logistic regression and determine the best metric to use for further refining the model.

### Solution Statement

The code for the base pipeline is given below.  The four key funtion serve the following functions in this step:

    load_data_from_database | Accesses the database and save the data from the 'dsi' table in 'data'
                            |
    make_data_dict          | Generates features and labels, then splits into training and validation sets
                            |
    general_transformer     | Normalize the features on the training set, then apply that transformation to the
                            | validation set
                            |
    general_model           | Fit and score a logistic regression given the data sets an a few inputs

In [11]:
from lib.project_5 import load_data_from_database, make_data_dict, general_model, general_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model.logistic import LogisticRegression

data = load_data_from_database(local = True)
data_dict = make_data_dict(data, rand_seed = 742)
data_dict = general_transformer(StandardScaler(), data_dict)
unregularized_logistic = LogisticRegression(penalty='l2',solver='liblinear',fit_intercept=True,C=1e10)
data_dict = general_model(data_dict, unregularized_logistic, test_scores=False)

To interpret the statistical metrics, the baseline accuracy must be considered.  The baseline accuracy and the calculated metrics are given below.

In [12]:
print "Baseline Accuracy:",max([data_dict['y_train'].mean(),1-data_dict['y_train'].mean()])
data_dict['metrics'][0]

Baseline Accuracy: 0.500714285714


,Score,Cross-Validation,Validation
0,accuracy,0.536383,Too Early for Validation
1,roc_auc,0.535550,Too Early for Validation
2,precision,0.538008,Too Early for Validation
3,recall,0.534909,Too Early for Validation
4,f1,0.536254,Too Early for Validation


The accuracy of the naive logistic fit is only slightly higher than the baseline accuracy, and the other metrics are similarly poor.  A significant amount of work will be required to generate an predictive regression.

### Metric

I will consider the accuracy as the significant metric for this project.  Since the baseline accuracy is nearly 50%, this means that the test training set has a nearly equal number of positive and negative labels.  As such, all the significance metrics would be expected to increase with one another (in contrast to scenarios where one label significantly outnumbers another, where accuracy can increase but precision and/or recall can be nearly zero as the model approaches the situation of uniformly predicting one class).  Moreover, since the data set is synthetic, there is no obvious metric which is inherently desireable for field-specific reason.  In this scenario, accuracy is the easiest metric to extract meaning from, so I will use it throughout.

### Benchmark

The benchmark for learning upon a data set requires an understanding of the baseline accuracy and the predictive ability of a simple model.  The baseline accuracy informs process by determining the appropriate metrics -- accuracy is sufficient for approximately equally-distributed classes which represent large portions of the data set, while other metrics may be appropriate for a large number of classes or classes distributed in a highly unequal manner.  The simplistic model predictivity gives a sense of how much feature/model selection may be necessary to improve the result.

In this case the benchmark accuracy is 53.6%, only marginally larger than the baseline accuracy of 50%, so significant improvements must be made to produce even a moderately predictive model.

## Implementation

Implement the following code pipeline using the functions you write in `lib/project_5.py`.

<img src="assets/benchmarking.png" width="600px">